# Data Collection
Both data sources need subscriptions, no fees needed tho
- Trade data come from BACI, a modified version of UNCOMTRADE data
    - BACI does not provide service trade, only UNCOMTRADE does
- Tariff data come from WITS TRAIN, use _Tariff and Trade Analysis_ under the _Advanced Query_ tab to get Effective Applied Tariff data
    - https://wits.worldbank.org/wits/wits/witshelp/content/data_retrieval/p/intro/c2.types_of_tariffs.htm

In [2]:
import pandas as pd 
import numpy as np
import os
import re
import copy
import time
import csv

# BACI
### Country Concordance
- Trade data of ‘Asia, not elsewhere specified’ (iso 490), is a good proxy for trade of Taiwan. In theory trade data for territories belonging to Asia, but not specified by country, could end up in this category, but in practice, only trade of Taiwan is included under this code, except for a couple of reporters. Use 490 as a proxy.
   
    
-  <font color='red'> Note that starting in the 2024Jan release, BACI adopted country spellings from UNComtrade which marks the end year of those dissovled countries or unions that are not used for reporting
    - 58 Belgium-Luxembourg (...1998); Belgium and Luxembourg are aggregated in BACI, because some countries do not distinguish between them in their declarations. The BACI procedure reconciles trade declaration of importers and exporters, so we have to aggregate the two countries in order to use all the trade declarations. Use their GDP to disaggrate.
    - 711 Southern African Customs Union (...1999); In BACI, we aggregate SACU countries in order to make the harmonization of imports and exports declarations. Indeed, some countries do not report trade with each SACU country individually, but with SACU as a whole. It means that Botswana, Lesotho, Namibia, South Africa and Eswatini (Swaziland) are recorded as SACU (code 711). Use each country's GDP to disaggrate.

    
- Trade values for some countries include more territories
    - USA, Puerto Rico and US Virgin Islands
        - starting in 2024 version, BACI just says USA; It's trade is facilitated by US customs system.
    - Switzerland, Liechtenstein
    - France, Monaco 
        - starting in 2024 version, BACI just says France;
    - As always, IFs do not put Macao under Mainland, China

In [3]:
### get a set of all product codes as well as country codes in BACI
h0_baci = set()
c_baci = set()
baci_list = os.listdir("data/BACI_HS92_V202401b/")
years = [str(y) for y in range(1995,2023)]
for y in years:
    dt = pd.read_csv(f"data/BACI_HS92_V202401b/BACI_HS92_Y{y}_V202401b.csv", encoding = "latin-1")
    h0_baci = h0_baci | set(dt.k)
    c_baci = c_baci | set(dt.i) | set(dt.j)
    #dt_sample = dt.iloc[:1000,].copy()
del dt

In [6]:
### concordance for country names
baci_country = pd.read_csv("data/BACI_HS92_V202401b/country_codes_V202401b.csv", encoding = "latin-1")
ifs_country = pd.read_excel("concordance/ccode_Comtrade_BACI_WITS_20241113.xlsx")
ifs_country = ifs_country[["BACI","Country"]].dropna()
# for c in ifs_country.BACI:
#     if c not in baci_country.country_name.unique():
#         print(c)
baci_ifs = dict(zip(ifs_country.BACI, ifs_country.Country))
baci_code_to_name = dict(zip(baci_country.country_code, baci_country.country_name))
# baci_country["Country_IFs"] = baci_country["country_name_full"].map(baci_ifs)
# baci_ifs = baci_country[["country_code", "Country_IFs"]].dropna()
# baci_ifs =  dict(zip(baci_ifs.country_code, baci_ifs.Country_IFs))

### Product Code
- Code sequence, 1988/92 H0, 1996 H1, 2002 H2, 2007 H3, 2012 H4, 2017 H5, 2022 H6...

- HS1992(HS0) was used in the BACI 
    - HS codes in actual data are read in as integer (unless there are commodities '9999AA'), so are the HS codes in the concordance table
    - HS codes read from the BACI product code book are text due to '9999AA'
    - Verify that we don't miss any commodity
    - In the file of BACI HS92(HS0) Code, it included codes from other HS versions! However, the actual data might not contain any code outside of H0 version

- GTAP 9 & GTAP 10 have different sectors, while GTAP 10 & 11 are the same


In [7]:
# mapping from hs combined to h0 to GTAP to IFs
h0_gtap = pd.read_csv("concordance/hs92gtap.csv", encoding="latin-1")
h0_gtap = h0_gtap[[" Product Code", "GTAP Product Code"]]
h0_gtap.columns = ["H0", "GTAP"]
h0_gtap = dict(zip(h0_gtap.H0, h0_gtap.GTAP))
gtap_ifs = pd.read_excel("concordance/GTAP 9_10_11 to IFs.xlsx")
gtap_ifs = gtap_ifs[["10number", "GTAP10toIFs"]]
gtap_ifs.columns = ["GTAP", "IFs"]
gtap_ifs = dict(zip(gtap_ifs.GTAP, gtap_ifs.IFs))
# Ensure there is no unidentified commodities
for h in h0_baci:
    assert h in h0_gtap
baci_prod =pd.read_csv("data/BACI_HS92_V202401b/product_codes_HS92_V202401b.csv",encoding="latin1")
baci_prod = set(baci_prod.code)
baci_prod_int = {int(c) for c in baci_prod if c!='9999AA'}
for c in h0_baci:
    assert c in baci_prod_int

### Unit Conversion
Data are in Current USD, convert to 2011USD or 2017USD
- Use GDP deflator from https://data.worldbank.org/indicator/NY.GDP.DEFL.ZS, note that this is not the right way to calcuate a "constant price" value
- In the furture, it is better to use % of GDP

In [8]:
deflator=pd.read_csv("concordance/WDI GDP Deflator 20241119.csv", skiprows=3)
deflator=deflator[deflator["Country Name"]=="United States"][[*["Country Name"], *[str(y) for y in range(1995,2023)]]].copy(True)

# WITS
### Country concordance
- Table formats are similar to https://wits.worldbank.org/wits/wits/witshelp/Content/Data_Retrieval/P/AQ/C9e.AQ_Tariffs-Result.htm
    - Reporter is importer, Partner is exporter
    - EU and World Tariff can be used to fill holes


In [15]:
c_wits = set()
for year in range(1995,2023):
    dt = pd.read_csv(f"data/WITS_Trade&Tariff_EffectiveApplied_20250117/split/{year}.csv", encoding="latin-1", 
                usecols = ['Product', "Product Name", 'DutyType','Reporter Name','Partner Name'])
    c_wits = c_wits | set(dt["Partner Name"]) | set(dt["Reporter Name"])
del dt

In [29]:
ifs_country = pd.read_excel("concordance/ccode_Comtrade_BACI_WITS_20241113.xlsx")
ifs_country = ifs_country[["WITS","Country","EU1995",'EU2004', 'EU2007', 'EU2013', 'EU2020']].dropna(subset=["WITS"]) 
# creating different lists is due to the changes of EU membership
EU1995 =list(ifs_country[ifs_country["EU1995"]==1].Country)
EU2004 =list(ifs_country[ifs_country["EU2004"]==1].Country)
EU2007 =list(ifs_country[ifs_country["EU2007"]==1].Country)
EU2013 =list(ifs_country[ifs_country["EU2013"]==1].Country)
EU2020 =list(ifs_country[ifs_country["EU2020"]==1].Country)
eu_cdict = {1995: EU1995, 2004: EU2004, 2007: EU2007, 2013: EU2013, 2020: EU2020}
wits_ifs = dict(zip(ifs_country.WITS, ifs_country.Country))
# for c in c_wits:
#     if c not in ifs_country.WITS.unique():
#         print(c.__repr__())

### Product Code
- Code sequence, 1988/92 H0, 1996 H1, 2002 H2, 2007 H3, 2012 H4, 2017 H5, 2022 H6...
- HSCombined was used in the WITS
    - WITS TRAINS has a tool under their "SUPPORT MATERIALS" to query for HS code conversion
    - its H92TOGTAP conversion is based on GTAP10
- Errors in the raw files
    - In WITS' mapping files, their HScombinehs92 is missing some products, e.g. 30249 from H5 is in the actual data file, marked as both HS & H5, but not in the HStoH0 mapping
    - Across different mapping files, there could be duplicates, e.g., 10130 from H4 is 10120 in H0, while 10130 from h5 is 10111 in H0
    - In WITS' raw data, their Native Nomen could be wrong, e.g., 271000 was reported between EU & Zambia as H5, but it should be H1 
    - <font color='red'> Hence, without merging on the column "Native Nomen", we might end up with multiple H0 on same reporting, we will take the minimal

In [28]:
hs_list = []
for f_hs in os.listdir("concordance"):
    if f_hs.endswith("hs92.csv"):
        dt = pd.read_csv(f"concordance/{f_hs}",encoding="latin-1")
        dt.drop(columns=["HS 1988/92 Product Description"], inplace=True)
        dt["Native Nomen"] = f_hs.replace("hs92.csv", "")
        hs_list.append(dt)
hs_list = pd.concat(hs_list)
hs_list.columns=["Product", "Product Description", "H0", "Native Nomen"]
hs_list = hs_list.sort_values(by=["Product", "Native Nomen"]).reset_index(drop=True)
hs_list["Native Nomen"] = hs_list["Native Nomen"].map({
    'hs96':"H1",'hs02': "H2", 'hs07':"H3", 'hs12':"H4", 'hs17':'H5', 'hs22':'H6', "hscombine":"HS"
})
### check for duplicates
# hs_list_check = hs_list.groupby(["Product"])["H0"].nunique().reset_index()
# hs_list_check[hs_list_check.H0 != 1]
# hs_list[hs_list.Product.isin([10130, 10190])]

### Duty type
- **AHS for effectively applied tariffs**, https://wits.worldbank.org/wits/wits/witshelp/content/data_retrieval/p/intro/c2.types_of_tariffs.htm
    - BND for bound tariffs, MFN for MFN applied tariffs, PRF for preferential tariffs
    - WITS uses the concept of effectively applied tariff which is defined as the lowest available tariff. If a preferential tariff exists, it will be used as the effectively applied tariff. Otherwise, the MFN applied tariff will be used.
- Table formats are similar to https://wits.worldbank.org/wits/wits/witshelp/Content/Data_Retrieval/P/AQ/C9e.AQ_Tariffs-Result.htm
    - Simple average: The 6-digit tariff is itself an average of included tariff line level lines.
- Tariff can be NULL for some country pairs
    - Potential solution is to custom union to fill in values (e.g., EU for EU members)

In [40]:
def tariff_save(y_start = 1995, y_end=2004, eu_clist_dict = eu_cdict):
    eu_clist = eu_cdict[y_start]
    start = time.time()
    for year in range(y_start,y_end+1):
        wits = pd.read_csv(f"data/WITS_Trade&Tariff_EffectiveApplied_20250117/split/{year}.csv", encoding="latin-1", 
                        usecols = ['Product', "Product Name", 'Reporter Name','Partner Name',  'Simple Average'])
        wits["Importer"]=wits["Reporter Name"].map(wits_ifs)
        wits["Exporter"]=wits["Partner Name"].map(wits_ifs)
        wits=wits.dropna(subset=["Importer","Exporter"],how="any").drop(columns=['Reporter Name','Partner Name'])
        #
        wits = pd.merge(left=wits, right = hs_list, on =["Product"], how="left")
        assert wits[wits["H0"].isna()].empty
        wits = wits.groupby(["Importer","Exporter","H0"],as_index=False)["Simple Average"].agg("min")
        wits.sort_values(by=["Importer","Exporter","H0"],inplace=True)
        wits.reset_index(drop=True,inplace=True)
        # separate EU into individual entries to fillin NULLs
        wits_eu=[]
        for c in eu_clist:
            wits_eu_im = wits[wits["Importer"].isin(["European Union"])].copy()
            wits_eu_im["Importer"] = c
            wits_eu_ex = wits[wits["Exporter"].isin(["European Union"])].copy()
            wits_eu_ex["Exporter"] = c
            wits_eu.append(wits_eu_im)
            wits_eu.append(wits_eu_ex)
        del wits_eu_im, wits_eu_ex
        wits_eu=pd.concat(wits_eu)
        # WITS without EU
        wits = wits[ ~(wits["Importer"].isin(["European Union"]) |  wits["Exporter"].isin(["European Union"]))]
        # subset of WITS without EU as a whole but has individual EU members
        wits_1 = wits[(wits["Importer"].isin(eu_clist))   | (wits["Exporter"].isin(eu_clist))]
        # subset of WITS without EU as a whole OR individual EU members
        wits_2 = wits[~((wits["Importer"].isin(eu_clist))   | (wits["Exporter"].isin(eu_clist)))]
        del wits
        # Avg_x is the tariff from EU as a whole, Avg_y is the tariff from individual EU members
        wits_eu=pd.merge(left=wits_eu,right=wits_1,on=["Importer","Exporter","H0"],how="outer")
        del wits_1
        wits_eu['Simple Average_y'] = wits_eu['Simple Average_y'].fillna(wits_eu['Simple Average_x'])
        wits_eu.rename(columns={'Simple Average_y':"Simple Average"},inplace=True)
        wits_eu.drop(columns=['Simple Average_x'],inplace=True)
        wits = pd.concat([wits_eu, wits_2],sort=False)
        wits.sort_values(by=["Importer","Exporter","H0"],inplace=True)
        wits.reset_index(drop=True,inplace=True)
        wits.columns=["Importer","Exporter","Commodity","TariffPct"]
        wits.to_csv(f"output/tariff_ifscountry/{year}.csv", index=False, encoding="latin-1")
    del wits
    end = time.time()
    print((end-start)/60)
eu_cdict.keys()

dict_keys([1995, 2004, 2007, 2013, 2020])

In [41]:
tariff_save(1995,2003)
tariff_save(2004,2006)
tariff_save(2007,2012)
tariff_save(2013,2022)

5.698436407248179
5.285443576176961
7.853725600242615
12.853086706002554


# Trade, Rev, and Weighted Tariff
- Missing Tariff 
    - use avg by importer & commodity
    - use avg by commodity
    - fill with 0s
- Tariff & Weighted Tariff are in %
- Trade needs to be converted to 2011 USD/2017USD, so does Rev
- Trade needs to be in different scale, so does Rev (*1000)

In [3]:
deflator=pd.read_csv("concordance/WDI GDP Deflator 20241119.csv", skiprows=3)
deflator=deflator[deflator["Country Name"]=="United States"][[*["Country Name"], *[str(y) for y in range(1995,2023)]]].copy(True)

In [4]:
start = time.time()
for year in range(1995,2023):
    # read in merge trade & tariff
    trade = pd.read_csv(f"output/trade_ifscountry/{year}.csv", encoding = "latin-1")
    # convert to $2017 & multiply by 1000
    trade["Trade"] = 1000*trade["Trade"]*deflator["2017"].values[0]/deflator[str(year)].values[0]
    tariff = pd.read_csv(f"output/tariff_ifscountry/{year}.csv", encoding = "latin-1")
    trade = pd.merge(left=trade, right=tariff, on = ["Exporter", "Importer", "Commodity"], how = "left")
    # fill in missing tariff by order
    tariff_by_importer = trade.groupby(["Importer", "Commodity"], as_index=False)["TariffPct"].agg("mean")
    tariff_by_importer.rename(columns={"TariffPct":"TariffPct_AvgIm"},inplace=True)
    tariff_by_commodity = trade.groupby(["Commodity"], as_index=False)["TariffPct"].agg("mean")
    tariff_by_commodity.rename(columns={"TariffPct":"TariffPct_AvgProd"},inplace=True)
    trade = pd.merge(left=trade, right=tariff_by_importer, on=["Importer", "Commodity"], how = "left")
    trade = pd.merge(left=trade, right=tariff_by_commodity, on=["Commodity"], how = "left")
    del tariff_by_importer, tariff_by_commodity
    trade["TariffPct"] = trade["TariffPct"].fillna(trade["TariffPct_AvgIm"]) 
    trade["TariffPct"] = trade["TariffPct"].fillna(trade["TariffPct_AvgProd"]) 
    trade["TariffPct"] = trade["TariffPct"].fillna(0)
    trade.drop(columns=["TariffPct_AvgIm", "TariffPct_AvgProd"], inplace=True)
    # Rev
    trade["Rev"]=trade["Trade"]*trade["TariffPct"]/100
    # Wt Tariff
    wt_all = trade.groupby(["Exporter","Importer"],as_index=False)[["Trade","Rev"]].agg("sum")
    wt_all["IFsSector"] = "AllGoods"
    wt_sector = trade.groupby(["Exporter","Importer","IFsSector"],as_index=False)[["Trade","Rev"]].agg("sum")
    wt_all=pd.concat([wt_all,wt_sector],sort=False)
    del trade, wt_sector 
    wt_all["WtTariff"]=(wt_all["Rev"]/wt_all["Trade"])*100
    wt_all.to_csv(f"output/trade_wt_rev_2017usd_ifscountry/{year}.csv", index=False, encoding = "latin-1")
del wt_all
end = time.time()
print((end-start)/60)

10.388817965984344
